In [6]:
import cv2
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
import random
import os
from PIL import Image



#이미지 증강
def img_res(im,txt):
    total=[] # 변환된 라벨 저장
    bbox=[] # 라벨 박스 저장
    img = cv2.imread(im) # 이미지 불러오기
    gt = np.loadtxt(txt, dtype = float, delimiter = ' ') # 라벨 값 불러오기
    img_x, img_y = img.shape[1],img.shape[0] # 이미지 사이즈 저장
    
    # 라벨이 없다면, 이미지만 return
    if gt.shape[0]==1:
        return img, ""
    
    input_img = img[np.newaxis, :, :, :]


    # bounding box 저장
    for i in range(gt.shape[0]):
        bbox.append(ia.BoundingBox(x1 = gt[i][1]*img_x-(gt[i][3]*img_x)/2, y1 = gt[i][2]*img_y-(gt[i][4]*img_y)/2, x2 = gt[i][1]*img_x+(gt[i][3]*img_x)/2, y2 = gt[i][2]*img_y+(gt[i][4]*img_y)/2))

    #augmentation 방식 지정(imgaug 사용)
    seq = iaa.Sequential([
            iaa.Multiply((1.2, 1.5)),
            iaa.Affine(
                scale = (0.9),
                rotate = (-5,5)),
        iaa.MultiplyAndAddToBrightness(mul=(0.6,1.4), add=0) # 밝기 조정
#        iaa.MotionBlur(k=(4,8)) # motionblur
#         iaa.HistogramEqualization() # HistogramEqualization 
#        iaa.Sharpen(alpha=0.5, lightness=1.0) # 선명도
    ])
    
    #변환된 이미지와 bounding box 생성
    aug_img, aug_bbox = seq(images = input_img, bounding_boxes = bbox)

    # 이미지의 변환된 라벨값을 텍스트로 저장
    for k in range(gt.shape[0]):
        box_list=[]
        box_list2=[]
        result = ' '.join(str(s) for s in [aug_bbox[k]])
        box_split = result.split(',')
        box_list.append(box_split[0].split('='))
        box_list.append(box_split[1].split('='))
        box_list.append(box_split[2].split('='))
        box_list.append(box_split[3].split('='))
        box_list2.append((float(box_list[0][1])+float(box_list[2][1]))/2/img_x)
        box_list2.append((float(box_list[1][1])+float(box_list[3][1]))/2/img_y)
        box_list2.append((float(box_list[2][1])-float(box_list[0][1]))/img_x)
        box_list2.append((float(box_list[3][1])-float(box_list[1][1]))/img_y)
        re=str(box_list2[0])+" "+str(box_list2[1])+" "+str(box_list2[2])+" "+str(box_list2[3])
        total.append(re)
    
    return aug_img[0],total



root_dir = "C:/scr/darknet/gukhap" # 디렉토리

possible_img_extension = ['.jpg', '.jpeg', '.JPG', '.bmp', '.png'] # 이미지 확장자들

fi_name = "re2" #파일 이름

os.mkdir("C:/scr/darknet" + "/"+ fi_name) #파일 생성


for (root, dirs, files) in os.walk(root_dir):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                box_list=[]
                img_path = root + '/' + file_name
                img_path = img_path.replace('\\', '/') # \는 \\로 나타내야함


                img, box = img_res(img_path,img_path[:-4]+'.txt') # 변환된 이미지 & 라벨 생성

                #라벨이 없는 것은 증강시키지 않는다
                if box=="":
                    continue
                
                #이미지와 라벨 저장
                cv2.imwrite("C:/scr/darknet/"+fi_name+"/aug_1_"+file_name,img)
                f1 = open("C:/scr/darknet/"+fi_name+"/aug_1_"+file_name[:-4]+".txt",'w')
                f2 = open("C:/scr/darknet/gukhap/"+file_name[:-4]+".txt",'r')
                j=0
                while True:
                    line = f2.readline()
                    if not line: break
                    f1.write((str(line[0])+str(line[1])).strip()+" "+box[j]+'\n')
                    j+=1
                f1.close()
                f2.close()
                
                
    


C:\Users\user\AppData\Local\Temp\ipykernel_4056\901022933.py:14: UserWarning: loadtxt: input contained no data: "C:/scr/darknet/build/darknet/x64/data/test/230630_pl_1063.txt"
  gt = np.loadtxt(txt, dtype = float, delimiter = ' ')
